# Maestría en "Analítica de datos"

* Nombre: Edisson Manuel Mora Madrigal

* Correo:

* Fecha: 2025-10-16

----
1. NODOS
* (n) -> un nodo cualquiera
* (:Persona) -> un nodo con etiqueta persona
* (:Persona {nombre: 'Luis'}) -> un nodo con la propiedad "nombre" que sea igual a "Luis"

2. REPRESENTACIONES
* -[r]- -> una relación cualquiera
* -[:TRABAJA_EN]- -> una relación de tipo "TRABAJA_EN"
* -[:AMIGO_DE]- -> una relación de tipo "AMIGO_DE"
* <-[:COMPRO]- => una relación entrante de tipo "COMPRO"

3. PATRONES DE CONSULTA
* MATCH (a:Persona)-[:AMIGO_DE]->(b:Persona) RETURN a, b



# 0. Instalar librerias

In [1]:
!pip install neo4j pandas matplotlib networkx pyvis -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.5 MB/s eta 0:00:00


# 1. Instanciar librerias

In [2]:
import pandas as pd
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network
from IPython.display import display, HTML

# 2. Configurar la conexión con Neo4j

In [3]:
NEO4J_URI = "neo4j+s://7f3373c9.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "yMYJXTugtfDEtwetK9XUvB0cPYYVmTz9lbKT7v2Sz7I"
NEO4J_DATABASE = "neo4j"

In [5]:
# clase para manejar la coenxión con NEO4j
class Neo4jConnection:
    def __init__(self, uri, user, password, database):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.database = database

    def close(self):
        self.driver.close()

    def query(self, query, parameters=None):
        with self.driver.session(database=self.database) as session:
            result = session.run(query, parameters)
            return [record for record in result]

    def execute(self, query, parameters=None):
        with self.driver.session(database=self.database) as session:
            session.run(query, parameters)


In [ ]:
def limpiar_bd_neo4j(con):
    con.execute("MATCH (n) DETACH DELETE n")

In [6]:
con = Neo4jConnection(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD, NEO4J_DATABASE)
print("Conexión establecida con NEO4J")


Conexión establecida con NEO4J


# Ejercicios

## Ejercicio 1: Red de Transmilenio

In [9]:
estaciones_data = [
    {"nombre": "Portal Norte", "zona": "Norte", "tipo": "Portal"},
    {"nombre": "Calle 100", "zona": "Norte", "tipo": "Intermedia"},
    {"nombre": "Calle 85", "zona": "Norte", "tipo": "Intermedia"},
    {"nombre": "Calle 72", "zona": "Centro", "tipo": "Intermedia"},
    {"nombre": "Heroes", "zona": "Centro", "tipo": "Troncal"},
    {"nombre": "Museo Nacional", "zona": "Centro", "tipo": "Intermedia"},
    {"nombre": "Av. Jimenez", "zona": "Centro", "tipo": "Troncal"},
    {"nombre": "Terrenos", "zona": "Sur", "tipo": "Intermedia"},
    {"nombre": "Nariño", "zona": "Sur", "tipo": "Intermedia"},
    {"nombre": "Portal Usme", "zona": "Sur", "tipo": "Portal"}
]
for estacion in estaciones_data:
    query = f"CREATE (:Estacion {{nombre: '{estacion['nombre']}', zona: '{estacion['zona']}', tipo: '{estacion['tipo']}'}})"
    con.execute(query)
    print(f"Nodo creado '{estacion['nombre']}'")


Nodo creado 'Portal Norte'
Nodo creado 'Calle 100'
Nodo creado 'Calle 85'
Nodo creado 'Calle 72'
Nodo creado 'Heroes'
Nodo creado 'Museo Nacional'
Nodo creado 'Av. Jimenez'
Nodo creado 'Terrenos'
Nodo creado 'Nariño'
Nodo creado 'Portal Usme'


In [11]:
# Crear las conexiones
conexiones = [
    ("Portal Norte", "Calle 100", 5),
    ("Calle 100", "Calle 85", 3),
    ("Calle 85", "Calle 72", 4),
    ("Calle 72", "Heroes", 6),
    ("Heroes", "Museo Nacional", 3),
    ("Museo Nacional", "Av. Jimenez", 4),
    ("Av. Jimenez", "Terreros", 5),
    ("Terreros", "Nariño", 4),
    ("Nariño", "Portal Usme", 8)
]
for origen, destino, tiempo in conexiones:
    con.execute("""
    MATCH (a:Estacion {nombre: $origen})
    MATCH (b:Estacion {nombre: $destino})
    CREATE (a)-[:CONECTA_CON {tiempo_minutos: $tiempo}]->(b)
    CREATE (b)-[:CONECTA_CON {tiempo_minutos: $tiempo}]->(a)
    """, {"origen": origen, "destino": destino, "tiempo": tiempo})

print(f"Conexiones creadas: {len(conexiones)}")


Conexiones creadas: 9


## Consultas

In [12]:
print("Estaciones por zona")
resultado = con.query("""
MATCH (e:Estacion)
RETURN e.zona as Zona, count(e) as Cantidad
ORDER BY Zona
""")

for record in resultado:
    print(f"Zona: {record['Zona']}, Cantidad: {record['Cantidad']}")


Estaciones por zona
Zona: Centro, Cantidad: 4
Zona: Norte, Cantidad: 3
Zona: Sur, Cantidad: 3


In [13]:
print("Ruta más corta entre el portal del Norte y el portal Usme")
resultado = con.query("""
MATCH path = shortestPath(
  (inicio:Estacion {nombre: 'Portal Norte'})-[:CONECTA_CON*]-(fin:Estacion {nombre: 'Portal Usme'})
)
RETURN [n IN nodes(path) | n.nombre] as Ruta,
       length(path) as Paradas
""")

print(resultado)
for record in resultado:
    print(f"Ruta: {record['Ruta']}")
    print(f"Paradas: {record['Paradas']}")
    # print(f"Tiempo total: {record['tiempo_total']}")


Ruta más corta entre el portal del Norte y el portal Usme
[]


## Visualización del grafo